<a href="https://colab.research.google.com/github/SepehrShahsavar/LLM-Fine-tune-for-german-to-french-translation/blob/main/GenAIPortfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q datasets
!pip install -q huggingface_hub
!pip install -q transformers
!pip install -q peft
!pip install -q accelerate
!pip install -q evaluate
!pip install -q sacrebleu
!pip install -q -U bitsandbytes
!pip install -q trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
# First, mount Google Drive (if you haven't already in another cell)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import nbformat
import os


notebook_path = '/content/drive/MyDrive/Colab Notebooks/GenAIPortfolio.ipynb'

# Load and clean notebook
# Use notebook_path instead of notebook_filename
with open(notebook_path, 'r', encoding='utf-8') as f:
    nb = nbformat.read(f, as_version=4)

# Remove problematic metadata
if 'widgets' in nb['metadata']:
    del nb['metadata']['widgets']
    print("✅ Removed 'widgets' metadata.")
else:
    print("ℹ️ No 'widgets' metadata found.")

# Save the cleaned notebook
# Use notebook_path instead of notebook_filename
with open(notebook_path, 'w', encoding='utf-8') as f:
    nbformat.write(nb, f)

print(f"✅ Notebook '{notebook_filename}' cleaned and saved.")

✅ Removed 'widgets' metadata.
✅ Notebook 'GenAIPortfolio.ipynb' cleaned and saved.


In [ ]:
import getpass
import os
from huggingface_hub import login
if "HF_TOKEN" not in os.environ:
    os.environ["HF_TOKEN"] = getpass.getpass("Enter your Huggingfacehub api token: ")



Enter your Huggingfacehub api token: ··········


In [ ]:
from datasets import load_dataset

ds = load_dataset("Helsinki-NLP/opus-100", "de-fr", split="test")
print(len(ds))
print(ds[0]['translation']['de'])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/65.4k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/312k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

2000
Nun, das ist eine medizinisch unratsame Portion Eiscreme.


In [ ]:
import torch
from transformers import AutoTokenizer

def preprocess(dataset):
  """
  Processes a batch of examples by iterating through them individually.

  Args:
      dataset: A dictionary of lists representing a batch of data.

  Returns:
      A dictionary with the updated 'input_ids', 'attention_mask', and 'labels' for the entire batch.
  """
  # Initialize empty lists to store the tokenized data for the batch
  input_ids_list = []
  attention_mask_list = []
  labels_list = []

  for i in range(len(dataset['translation'])):
    prompt = (
          "Translate the following German sentence into French:\n"
          f"German: {dataset['translation'][i]['de']}\n"  # Access individual example
          "French:"
      )
    target = ' ' + dataset['translation'][i]['fr'] + tokenizer.eos_token # Access individual example
    full_text = prompt + target
    tokenized_full = tokenizer(
          full_text,
          return_tensors="pt",
          truncation=True,
          max_length=512,
          padding="max_length"
      )

    # Tokenize the prompt separately (without adding special tokens)
    # to find out where the target translation starts
    tokenized_prompt = tokenizer(prompt, add_special_tokens=False).input_ids
    prompt_length = len(tokenized_prompt)

    # Create a copy of input_ids to serve as labels
    labels = tokenized_full["input_ids"].clone()

    # Mask the prompt portion by setting labels to -100 up to prompt_length
    labels[:, :prompt_length] = -100

    #  mask padding tokens in labels
    # (assuming the tokenizer.pad_token_id is defined)
    labels = torch.where(labels == tokenizer.pad_token_id, -100, labels)

    # Append the tokenized data for the current example to the lists
    input_ids_list.append(tokenized_full["input_ids"][0])  # Assuming batch size of 1
    attention_mask_list.append(tokenized_full["attention_mask"][0])  # Assuming batch size of 1
    labels_list.append(labels[0])  # Assuming batch size of 1

  # Stack the tokenized data to create tensors for the batch
  input_ids_batch = torch.stack(input_ids_list)
  attention_mask_batch = torch.stack(attention_mask_list)
  labels_batch = torch.stack(labels_list)

  # Return a dictionary containing the tokenized data for the entire batch
  return {
      "input_ids": input_ids_batch,
      "attention_mask": attention_mask_batch,
      "labels": labels_batch,
  }

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
# Load model directly
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM ,BitsAndBytesConfig
from transformers import GPT2Tokenizer, GPT2Model


quant_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_use_double_quant= True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype= torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2-xl")
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2-xl", quantization_config = quant_config)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=16,
    lora_alpha=64,
    lora_dropout=0.1,
    target_modules=["c_attn"],
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [ ]:
split_dataset = ds.train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]
tokenizer.pad_token = tokenizer.eos_token
# Apply the preprocessing function to both splits
train_dataset = train_dataset.map(preprocess, batched=True)
test_dataset = test_dataset.map(preprocess, batched=True)

Map:   0%|          | 0/1800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
import evaluate

# Load the sacreBLEU metric
bleu_metric = evaluate.load("sacrebleu")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode the predictions and labels to text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Optional: Post-process to remove any prompt part if necessary.
    # Here we assume the model outputs only the translation text.

    # The sacreBLEU metric expects a list of references for each prediction
    references = [[ref] for ref in decoded_labels]
    result = bleu_metric.compute(predictions=decoded_preds, references=references)

    # Return the BLEU score (commonly as a percentage)
    return {"bleu": result["score"]}


In [ ]:
import torch
from tqdm import tqdm
import evaluate

def evaluate_model(model, tokenizer, test_dataset, device, max_new_tokens=128):
    # Load sacreBLEU metric
    bleu_metric = evaluate.load("sacrebleu")

    model.eval()
    predictions = []
    references = []

    # Iterate over the test examples
    for example in tqdm(test_dataset, desc="Evaluating"):
        # Build the prompt using the consistent format.
        prompt_text = (
            "Translate the following German sentence into French:\n"
            f"German: {example['translation']['de']}\n"
            "French:"
        )

        # Tokenize the prompt text.
        inputs = tokenizer(prompt_text, return_tensors="pt", truncation=True, max_length=512)
        inputs = inputs.to(device)

        # Generate the model's output.
        with torch.no_grad():
            output_ids = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=0.7,         # Adjust as needed
                top_k=50,
                top_p=0.95,
                repetition_penalty=1.2,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )

        # Decode the full output.
        output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        # Extract the translation part by splitting at "French:".
        if "French:" in output_text:
            generated_translation = output_text.split("French:")[-1].strip()
        else:
            generated_translation = output_text.strip()

        predictions.append(generated_translation)
        # sacreBLEU expects a list of reference lists per prediction.
        references.append([example['translation']['fr']])

    # Compute BLEU score.
    results = bleu_metric.compute(predictions=predictions, references=references)
    print("BLEU score:", results["score"])
    return results


In [ ]:
# Assuming 'test_dataset' is your test split and 'model' & 'tokenizer' are loaded,
# and that you have set your device accordingly.
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
results = evaluate_model(model, tokenizer, test_dataset, device)

Evaluating: 100%|██████████| 200/200 [21:16<00:00,  6.38s/it]

BLEU score: 0.10441847614655263


In [ ]:
#test model output for a sample
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
prompt = """### Translation Task ###
Input (German): Nun, das ist eine medizinisch unratsame Portion Eiscreme.
Output (French):"""


inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(model.device)

with torch.no_grad():
  outputs = model.generate(**inputs,
                           temperature=0.7,
                           top_k=50,
                           top_p=0.95,
                           repetition_penalty=1.2,
                           do_sample=True,
                           max_new_tokens=128,
                           pad_token_id=tokenizer.eos_token_id,
                           eos_token_id=tokenizer.convert_tokens_to_ids("<EOS>")
                           )
prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(prediction)

### Translation Task ###
Input (German): Nun, das ist eine medizinisch unratsame Portion Eiscreme.
Output (French): Nous voudrons avec les mains qui nous ont été prêts de l'abord.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
for name, param in model.named_parameters():
    if "lora" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False  # Ensure base model is frozen

model.print_trainable_parameters()

trainable params: 4,915,200 || all params: 1,562,526,400 || trainable%: 0.3146


In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

1978

In [ ]:
PATH = "/content/drive/MyDrive/Model-A" # Assuming 'Model-A' is the directory
model = AutoModelForCausalLM.from_pretrained(PATH, quantization_config=quant_config) # Load the model using from_pretrained

`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [ ]:
from transformers import TrainerCallback, TrainerControl, TrainerState, TrainingArguments
import torch
import gc

class MemoryCleanupCallback(TrainerCallback):
    def on_step_end(self, args: TrainingArguments, state: TrainerState, control: TrainerControl, **kwargs):
        """
        Event called at the end of a training step.
        """
        # Perform garbage collection and clear CUDA cache
        gc.collect()
        torch.cuda.empty_cache()
        return control

    def on_epoch_end(self, args: TrainingArguments, state: TrainerState, control: TrainerControl, **kwargs):
        """
        Event called at the end of an epoch.
        """
        # Perform garbage collection and clear CUDA cache
        gc.collect()
        torch.cuda.empty_cache()
        return control




In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir=PATH,
    per_device_train_batch_size=2,         # Adjust based on GPU memory
    gradient_accumulation_steps=32,          # To simulate a larger batch size if needed
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=10,
    fp16=True,                             # Use if supported by your GPU
    evaluation_strategy="steps",
    eval_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True,
    report_to="tensorboard",
    gradient_checkpointing=True,
)

memory_cleanup_callback = MemoryCleanupCallback()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[memory_cleanup_callback]
)



/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Start training
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


AssertionError: No inf checks were recorded for this optimizer.

In [ ]:
outputs = model(**batch)
loss = outputs.loss

# Ensure loss is not detached
print(loss.requires_grad)  # Should be True

loss.backward()

NameError: name 'batch' is not defined

In [ ]:
#save model parameters in drive
model.save_pretrained("/content/drive/MyDrive/Model-A")

In [ ]:
result = evaluate_model(model , tokenizer ,test_dataset , device)

Evaluating: 100%|██████████| 200/200 [25:16<00:00,  7.58s/it]

BLEU score: 0.4497678109120795


In [ ]:
#test model output for a sample
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
prompt = """### Translation Task ###
Input (German): Nun, das ist eine medizinisch unratsame Portion Eiscreme.
Output (French):"""


inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(model.device)

with torch.no_grad():
  outputs = model.generate(**inputs,
                           temperature=0.7,
                           top_k=50,
                           top_p=0.95,
                           repetition_penalty=1.2,
                           do_sample=True,
                           max_new_tokens=128,
                           pad_token_id=tokenizer.eos_token_id,
                           eos_token_id=tokenizer.convert_tokens_to_ids("<EOS>")
                           )
prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(prediction)

### Translation Task ###
Input (German): Nun, das ist eine medizinisch unratsame Portion Eiscreme.
Output (French): Cette partie s'applique a été une portée de bienvenu et une couteur éducative. J'ai été fait la version du porte. La section est étude par le loup. Pourquoi ? ! ?! ?!! Translation Task : Qui est quelque chose que la porte où elle nous suivra? qui se passe... Cela est en réaction aux mêmes temps. Et ça ? ! ?!!! ??? ?!!??? ??? ?? !!??? ??????? ??? ?


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-Coder-32B-Instruct")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-Coder-32B-Instruct", torch_dtype=torch.float16, device_map='auto')

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/63.2k [00:00<?, ?B/s]

model-00001-of-00014.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

model-00002-of-00014.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00003-of-00014.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

...

model-00012-of-00014.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00013-of-00014.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00014-of-00014.safetensors:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [ ]:
def generate_synthetic_translation():
  prompt = (
        "Generate a German sentence and its French translation.\n"
        "Format:\n"
        "German: <generated sentence>\n"
        "French: <translated sentence>\n\n"
        "Output:\n"
  )

  inputs = tokenizer(prompt, return_tensors='pt').to('cuda')

  with torch.no_grad():
    output = model.generate(
        **inputs,
        max_length=100,   # adjust as needed
        num_beams=1
        # skip_special_tokens = True
    )

  german_sentence = tokenizer.decode(output[0], skip_special_tokens=True).split("German:")[1].strip()
  french_translation = tokenizer.decode(output[0], skip_special_tokens=True).split("French:")[1].strip()

  return german_sentence, french_translation

In [ ]:
print(generate_synthetic_translation())

('<generated sentence>\nFrench: <translated sentence>\n\nOutput:', '<translated sentence>\n\nOutput:\nGerman: Ich liebe es, am Strand zu spazieren.')


In [ ]:
from transformers import pipeline

# Load model via Hugging Face API
generator = pipeline("text-generation", model="Qwen/Qwen2.5-Coder-32B-Instruct")

# Prompt for synthetic parallel data
prompt = (
    "Generate a German sentence and its French translation.\n"
    "Format:\n"
    "German: <generated sentence>\n"
    "French: <translated sentence>\n\n"
    "Output:\n"
)

# Generate text
output = generator(prompt, max_length=100)[0]['generated_text']
print(output)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/63.2k [00:00<?, ?B/s]

model-00001-of-00014.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

model-00002-of-00014.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

...

model-00011-of-00014.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00012-of-00014.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00013-of-00014.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00014-of-00014.safetensors:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]